In [5]:
import math as m
import numpy as np
import soundfile as sf
from dec2bin import *

v = 0
T = 1
A = 1 
dt = 0.1

inp = r'C:\Sem 5\Digital Communication\Labs\Lab 6\aotj1.wav'
out = r'C:\Sem 5\Digital Communication\Labs\Lab 6\of.wav'

data1, fs = sf.read(inp) 
data2 = data1[:160000]
data3 = np.unique(data1)
data4 = np.sort(data3) 

mstp = 5
for i in range(len(data4) - 1):
    if data4[i + 1] - data4[i] < mstp:
        mstp = data4[i + 1] - data4[i]
        
nol = int(2/mstp)
bpl = int(m.log(nol, 2))

hasht = {}
count = -1
b = 0
for i in range(nol):
    tmp = dec2bin(b).zfill(16)
    tmpstr = ' '.join(map(str, tmp)) 
    hasht.update({ tmpstr : count})
    b += 1
    count += mstp

bits = []
for i in range(len(data2)):
    minn = -1
    maxx =  1
    for j in range(bpl):
        mid = (minn + maxx)/2
        if data2[i][0] < mid:
            bits.append(0)
            maxx = mid
        if data2[i][0] >= mid:
            bits.append(1)
            minn = mid            

nsamp = int(T/dt)
s = m.sqrt(v)

s1 = []
for i in range(nsamp):
    s1.append(0)
for i in range(nsamp):
    s1.append(A)
for i in range(nsamp):
    s1.append(0)
    
s2 = []
for i in range(nsamp):
    s2.append(0)
for i in range(nsamp):
    s2.append(-A)
for i in range(nsamp):
    s2.append(0)
    
mf = np.subtract(s1,s2)

a1 = np.dot(s1,mf)
a2 = np.dot(s2,mf)
thr = (a1 + a2)/2

X = []
for i in range(len(bits)):
    if bits[i] == 1:
        for j in range(nsamp):
            X.append(A)
    if bits[i] == 0:
        for j in range(nsamp):
            X.append(-A)

awgn = (np.random.randn(len(bits)*nsamp))*s

R = np.add(X, awgn)

Z = np.convolve(mf, R)
Zsamp = []
j = 0
for i in range(len(Z)):
    if j == nsamp - 1:
        Zsamp.append(Z[i])
        j = 0
    else:
        j += 1

Rb = []
for p in range(len(Zsamp)):
    if Zsamp[p] < thr:
        Rb.append(0)
    if Zsamp[p] > thr:
        Rb.append(1)
        
Xr = []
i = 0
while i < len(Rb):
    bos = []
    for j in range(bpl):      
        bos.append(Rb[i])
        i += 1
    
    bosstr = ' '.join(map(str, bos))
    Xr.append(hasht[bosstr])

sf.write(out, Xr, fs)